In [ ]:
###############################################
'''
                 PIPE LINE 1
                 Data 전처리
              melspec computing
                                              '''
###############################################


''' 
    
    
    
데이터 셋이 너무 큰 경우 Part_id 로 쪼개서 작업할수도 있음
get_df 에서 학습용 df 전처리 하는게 유리할것. (df.rating이 일정이하면 학습에 안쓰겠다 등등)




'''



In [1]:
import numpy as np
import librosa as lb
import librosa.display as lbd
import soundfile as sf
from  soundfile import SoundFile
import pandas as pd
from  IPython.display import Audio
from pathlib import Path

import torch
from torch import nn, optim
from  torch.utils.data import Dataset, DataLoader
import torchvision.models as models

from matplotlib import pyplot as plt

import os, random, gc
import re, time, json
from  ast import literal_eval


from IPython.display import Audio
from sklearn.metrics import label_ranking_average_precision_score

from tqdm.notebook import tqdm
import joblib

import pretrainedmodels

from sklearn.model_selection import StratifiedKFold


In [2]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
device

'cuda'

In [3]:
##################
## Seed Setting ##
##################

def make_seed(seed = 499):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED'] = str(seed)
    
make_seed()

Num_classes = 397 ##label할 classes 갯수(ex 새 종류)
SR = 32000 #sampling rate
Duration = 7 #time
seed = 599
n_splits = 5

In [4]:
#################
## Path manage ##
#################

Root_Path = '/Users/ansgh/PycharmProjects/kaggle_notebook/input/birdclef-2021'

Train_short_root = Path(Root_Path + str('/train_short_audio'))
Train_metadata_root = Path(Root_Path + str('/train_metadata.csv'))

Train_audio_image_save_root = Path(Root_Path + str('/audio_images'))
Train_audio_image_save_root.mkdir(exist_ok=True, parents=True)

## File 내부 확인 
file_list = os.listdir(Root_Path)
print(file_list)

['audio_images', 'Label_IDS.json', 'resnet152_sr32000_d7_v1_v1', 'resnext_sr32000_d7_v1_v1', 'rich_train_metadata.csv', 'sample_submission.csv', 'test.csv', 'test_soundscapes', 'train_metadata.csv', 'train_short_audio', 'train_soundscapes', 'train_soundscape_labels.csv']


In [5]:
#####################
## Audio File info ##
#####################

def get_audio_info(filepath):
    with SoundFile(filepath) as f:
        sr = f.samplerate
        frames = f.frames
        duration = float(frames)/sr
        
    return {'frames': frames, 'sr':sr, 'duration': duration}

#오디오 파일에서 sr, running_time, frame 이 제일 중요한 정보기 때문에 이것들만 dic에 저장

In [6]:
###############################################
## 데이터셋이 클때 분할하기 위한 테크닉      ##
##   이거 한 후에 make_df함수에서 쓰면 됨    ##
###############################################

Part_id = 0
Part_indexs = np.linspace(0, 62874, 5)  #birdclef 가 62874개라 5분할
Part_indexs = list(map(int, Part_indexs)) #분할한후 index로 쓸것이기때문에 int로 치환

Part_indexs

[0, 15718, 31437, 47155, 62874]

In [7]:
################################################
##       Metadata + Audio_info 해서           ##
##              DataFrame 생성                ##
################################################

def make_df(n_splits = 5, seed = seed, nrows = None):
    
    df = pd.read_csv(Train_metadata_root, nrows=nrows) #전체열 다 불러옴
    Label_IDS = {label: label_id for label_id, label in enumerate(sorted(df['primary_label'].unique()))}  #고유종마다 ids 부여, ex 참새 = 0, 비둘기 = 1

    
    df = df.iloc[Part_indexs[Part_id] : Part_indexs[Part_id + 1]] #5분할중에 한파트
    df['label_id'] = df['primary_label'].map(Label_IDS) ##Primary Label을 기준으로 같은 종마다 같은 Label_IDS 배정 ex.참새_1 = 0, 참새_2 = 0, 비둘기 = 1 
    df['filepath'] = [str(Train_short_root/primary_label/filename) for primary_label, filename in zip(df.primary_label, df.filename)] #df에 file_path 추가

    
    pool = joblib.Parallel(6) ##병렬처리해서 속도up
    mapper = joblib.delayed(get_audio_info) #get_audio_info 함수
    tasks = [mapper(filepath) for filepath in df.filepath]
    # df의 filepath열에 저장된 filepath를 get_audio_info에 넣고 모든 병렬처리 끝날때까지 기다리도록 설정
    
    
    audio_info_index = []
    for i in range( Part_indexs[Part_id], Part_indexs[Part_id + 1] ):
        audio_info_index.append(i)
        
    df = pd.concat([df, pd.DataFrame( pool(tqdm(tasks)), index=audio_info_index )], axis=1, sort=False) #원래df 옆에 task한거(get_audio_info 결과) 붙임
    
    
    #나중에 K-Fold 쓰기위해서 Fold할 number를 df에 labeling
    skf = StratifiedKFold(n_splits=n_splits, random_state=seed, shuffle=True)
    splits = skf.split(np.arange(len(df)), y=df.label_id.values)
    df['fold'] = -1
    
    for fold, (train_set, val_set) in enumerate(splits):
        df.loc[df.index[val_set], 'fold'] = fold
    
    return Label_IDS, df

In [9]:
#######################################
##                                   ##  
## 음성처리 위한 Melspectrogram 계산 ## 
##                                   ##
#######################################

class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs['n_fft'] = kwargs.get('n_fft', self.sr//10) ##SR//10으로 n_fft 설정
        kwargs['hop_length'] = kwargs.get('hop_length', self.sr//(10*4)) ##hop length == fft마다 겹치는 길이
        self.kwargs = kwargs
    
    
    def __call__(self, audio):
        
        melspec = lb.feature.melspectrogram(
            audio, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs
        )
        melspec = lb.power_to_db(melspec).astype(np.float32)
        
        return melspec

In [10]:
#########################################
##          Mel Image normalize        ##   
## 이미지 노말라이징 안하면 너무 밝음  ##
#########################################

def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

In [11]:
######################################
##          Crop Padding            ##   
## 규격에 맞게 사이즈 trimming 하기 ##
######################################

def crop_or_pad(y, length, is_train=True, start=None):
    # y가 규격보다 짧으면
    if len(y) < length:
        y = np.concatenate([y, np.zeros(length - len(y))]) ## 부족한 길이를 0우로 padding 함
        
        n_repeats = length // len(y)
        epsilon = length % len(y)
        
        y= np.concatenate([y]*n_repeats + [y[:epsilon]]) ##적당히 채워줌
        
    # y가 규격보다 길면    
    elif len(y) > length:
        if not is_train:
            start = start or 0
        else:
            start = start or np.random.randint(len(y) - length)
            
        y = y[start : start + length] ##이러면 length짜리 y로 변경됨
        
        
    return y

In [12]:
############################                  
##   Audio to Image       ##
############################


class AudioToImage:
    def __init__(self, sr=SR, n_mels=128, fmin=0, fmax=None, duration=Duration, step=None, res_type='kaiser_fast', resample=True):
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2 ##안넣으면 nyquist로 씀
        self.duration = duration
        self.audio_length = self.duration * self.sr #audio들은 길이가 다 다르기 때문에 audio_length만큼씩 같은 길이로 토막내서 보관
        self.step = step or self.audio_length
        self.res_type = res_type
        self.resample = resample
        
        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax)
        
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio)
        image = mono_to_color(melspec)
        
        return image
    
    def __call__(self, row, save=True):
        audio, orig_sr = sf.read(row.filepath, dtype='float32')
        
        #전처리 Audio_to_img 하기전에 음성과 우리의 sr이 안맞으면 안되기때문에..
        if self.resample and orig_sr != self.sr: #resampling이 요청되거나(sampling rate가 안맞을때)
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        
        ##본처리
        
        #먼저 음성마다 다른 길이를 맞춰주기 위해서 같은 길이로 샘플링할것임
        # audio길이를 i:i+audio_len 즉 audio_len만큼 읽을건데, i범위는 0부터 총길이-원하는길이 , 원하는 길이 간격으로
        # 즉 audios에 저장 되는것은 [0~audio_len, audio_len~2*audio_len, ..... ,x*audio_len~총길이] 이런식으로 샘플링해서 저장
        audios = [audio[i: i+self.audio_length] for i in range(0, max(1, len(audio) - self.audio_length+1), self.step)]
        # 마지막 (x*audio_len ~ 총 길이) 인 샘플은 총 길이가 (x+1)*audio_len 이 아니면 더 짧게 나올것임, 따라서 crop_padding해줌
        audios[-1] = crop_or_pad(audios[-1], length=self.audio_length)
        
        #audios에 잘라서 넣은 audio들을
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
        
        
        #저장
        if save:
            path = Train_audio_image_save_root/f"{row.primary_label}/{row.filename}.npy"
            path.parent.mkdir(exist_ok=True, parents=True) #parent는 상위경로도 생성한다는 뜻
            np.save(str(path), images)
        
        else:
            return row.filename, images

In [13]:
## Audio to Image 실행

def get_audios_as_images(df):
    pool = joblib.Parallel(2)
    converter = AudioToImage(step=int(Duration*0.666*SR)) ##Step ==audio_length
    mapper = joblib.delayed(converter)
    tasks = [mapper(row) for row in df.itertuples(False)]

    pool(tqdm(tasks))

In [14]:
# 합친 Data_Frame 생성 (Part_id == 0 이기 때문에 5개를 loop로 5개 다 돌려야 전체 데이터 셋임)
for Part_id in range(4):
    Label_IDS, df = make_df()

    # 새 df를 csv로 저장
    df.to_csv(Root_Path + f'/rich_train_metadata_{Part_id}.csv', index=True)
    
    #json이 data 전송할때 더 유리하다고 함, json도 같이 저장해줍시다.
    with open(Root_Path + f'/Label_IDS_{Part_id}.json', 'w') as f: ## Label_IDS.json 이름의 파일을 'w'rite모드로 열어서
        json.dump(Label_IDS, f) #Label_IDS 객채를 json_file에 넣음
        
    get_audios_as_images(df) ##이건 넘어갔다가 Audio_do_image 파트 보고 다시올것.

  0%|          | 0/15718 [00:00<?, ?it/s]

  0%|          | 0/15718 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
'''
# For example.. we can see longest duration mel image
row_example = df.loc[df.duration.idxmax()]
mels_example = np.load(str((Train_audio_image_save_root/row.primary_label/row.filename).as_posix() + '.npy'))
print(mels_example.shape)
lbd.specshow(mels_example[0])
# as_posix 쓰면 str 안에 존재하는 \ 가 | 로 바뀜
'''